In [1]:
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook
from pennylane import numpy as np
import qiskit
import pennylane as qml
import random as rand
import time as time
import pandas as pd
import os as os
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.neural_networks import EffectiveDimension, LocalEffectiveDimension
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, RealAmplitudes
from typing import Optional, Union, List, Callable
from scipy.special import logsumexp


from platform import python_version

In [2]:
NUM_QUBITS = 4
NUM_LAYERS = 6
WEIGHTS_INIT = 0.01 * np.random.randn(NUM_LAYERS, NUM_QUBITS + 1, 3, requires_grad=True)
n_wires = 4
layers = 4
dev = qml.device('default.qubit', wires=4)
paths = ['iris.data', 'banknote.data']
DATAFRAMES = [((os.path.splitext(os.path.basename(path))[0]).capitalize(), pd.read_csv(
    path, names=['a0', 'a1', 'a2', 'aNUM_ITERATIONS', 'target'], header = 0)) for path in paths]



In [3]:
H = 1.*qml.PauliX(0) @ qml.PauliX(1) - 0.5 * qml.PauliZ(1)
@qml.qnode(dev)
def pennylane_circuit(weights, features):

    qml.AngleEmbedding(features=features, wires=range(4), rotation='Z')
    for W in weights:
        qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
        qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
        qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
        qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 0])
    return qml.expval(qml.PauliZ(0))

In [4]:
@qml.qnode(dev)
def hardware_efficient_circuit(weights, features):
    qml.AngleEmbedding(features=features, wires=range(4), rotation='Z')

    for W in weights[:-1]:
        qml.RY(W[0, 0], wires=0)
        qml.RZ(W[0, 1], wires=0)
        qml.RY(W[1, 0], wires=0)
        qml.RZ(W[1, 1], wires=0)
        qml.RY(W[2, 0], wires=0)
        qml.RZ(W[2, 1], wires=0)
        qml.RY(W[3, 0], wires=0)
        qml.RZ(W[3, 1], wires=0)
        qml.CZ(wires=[0, 1])
        qml.CZ(wires=[2, 3])
        qml.CZ(wires=[1, 2])

    W = weights[-1]
    qml.RY(W[0, 0], wires=0)
    qml.RZ(W[0, 1], wires=0)
    qml.RY(W[1, 0], wires=0)
    qml.RZ(W[1, 1], wires=0)
    qml.RY(W[2, 0], wires=0)
    qml.RZ(W[2, 1], wires=0)
    qml.RY(W[3, 0], wires=0)
    qml.RZ(W[3, 1], wires=0)

    return qml.expval(qml.PauliZ(0))

In [5]:
@qml.qnode(dev)
def pennylane_ab_parameterized_circuit(weights, features):
    qml.AngleEmbedding(features=features, wires=range(4), rotation='Z')

    for W in weights:
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 2])
        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 3])
        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 0])
        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 1])

    return qml.expval(qml.PauliZ(0))

In [6]:
@qml.qnode(dev)
def pairwise_controlled_rot_circuit(weights, features):
    qml.AngleEmbedding(features=features, wires=range(4), rotation='Z')

    for W in weights:
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 1])
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 2])
        qml.CRot(W[0, 0], W[0, 1], W[0, 2], wires=[0, 3])

        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 0])
        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 2])
        qml.CRot(W[1, 0], W[1, 1], W[1, 2], wires=[1, 3])

        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 3])
        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 1])
        qml.CRot(W[2, 0], W[2, 1], W[2, 2], wires=[2, 0])

        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 2])
        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 1])
        qml.CRot(W[3, 0], W[3, 1], W[3, 2], wires=[3, 0])

    return qml.expval(qml.PauliZ(0))


In [7]:
iris2 = np.array([[5.1,3.5,1.4],
[4.9,3.0,1.4,0.2],
[4.7,3.2,1.3,0.2],
[4.6,3.1,1.5,0.2],
[5.0,3.6,1.4,0.2],
[5.4,3.9,1.7,0.4],
[4.6,3.4,1.4,0.3],
[5.0,3.4,1.5,0.2],
[4.4,2.9,1.4,0.2],
[4.9,3.1,1.5,0.1],
[5.4,3.7,1.5,0.2],
[4.8,3.4,1.6,0.2],
[4.8,3.0,1.4,0.1],
[4.3,3.0,1.1,0.1],
[5.8,4.0,1.2,0.2],
[5.7,4.4,1.5,0.4],
[5.4,3.9,1.3,0.4],
[5.1,3.5,1.4,0.3],
[5.7,3.8,1.7,0.3],
[5.1,3.8,1.5,0.3],
[5.4,3.4,1.7,0.2],
[5.1,3.7,1.5,0.4],
[4.6,3.6,1.0,0.2],
[5.1,3.3,1.7,0.5],
[4.8,3.4,1.9,0.2],
[5.0,3.0,1.6,0.2],
[5.0,3.4,1.6,0.4],
[5.2,3.5,1.5,0.2],
[5.2,3.4,1.4,0.2],
[4.7,3.2,1.6,0.2],
[4.8,3.1,1.6,0.2],
[5.4,3.4,1.5,0.4],
[5.2,4.1,1.5,0.1],
[5.5,4.2,1.4,0.2],
[4.9,3.1,1.5,0.1],
[5.0,3.2,1.2,0.2],
[5.5,3.5,1.3,0.2],
[4.9,3.1,1.5,0.1],
[4.4,3.0,1.3,0.2],
[5.1,3.4,1.5,0.2],
[5.0,3.5,1.3,0.3],
[4.5,2.3,1.3,0.3],
[4.4,3.2,1.3,0.2],
[7.0,3.2,4.7,1.4],
[6.4,3.2,4.5,1.5],
[6.9,3.1,4.9,1.5],
[5.5,2.3,4.0,1.3],
[6.5,2.8,4.6,1.5],
[5.7,2.8,4.5,1.3],
[6.3,3.3,4.7,1.6],
[4.9,2.4,3.3,1.0],
[6.6,2.9,4.6,1.3],
[5.2,2.7,3.9,1.4],
[5.0,2.0,3.5,1.0],
[5.9,3.0,4.2,1.5],
[6.0,2.2,4.0,1.0]])

C:\Users\Carson\anaconda3\lib\site-packages\autograd\numpy\numpy_wrapper.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return _np.array(args, *array_args, **array_kwargs)


In [8]:
#Test circuit
@qml.qnode(dev)
def circ(params):
    qml.RY(params[0], wires=1)
    qml.CNOT(wires=(1,0))
    qml.RY(params[1], wires=1)
    qml.RZ(params[2], wires=1)
    return qml.expval(qml.PauliZ(0))

params = np.array([[5.1,3.5,1.4],[4.9,3.0,1.4],[4.7,3.2,1.3],[5.1,3.5,1.4],[4.9,3.0,1.4],[4.7,3.2,1.3],[4.9,3.1,1.5]], requires_grad=True)

In [9]:
circuit_lib = [circ]#, pennylane_circuit]#, hardware_efficient_circuit,
                    #pennylane_ab_parameterized_circuit, pairwise_controlled_rot_circuit]

In [10]:
def get_eff_dim(
        n: Union[List, int]
    ) -> Union[List[int], int]:    
    f_hat, trace = get_fhat(n)
    n = len(n)
            # step 4: calculate effective dimension(s) for each data sample size "n" out
            # of normalized fishers
    ns = n
    logsum_axis = None
    fs = f_hat * ns / (2 * np.pi * np.log(ns))
    one_plus_fs = np.eye(3) + fs
    dets = np.linalg.slogdet(one_plus_fs)[1]  # log det because of overflow
    rs = dets / 2  # divide by 2 because of sqrt
    effective_dims = 2 * (logsumexp(rs, axis=logsum_axis) - np.log(3)) / \
                        np.log(n / (2 * np.pi * np.log(n)))
    
    return np.squeeze(effective_dims)

In [11]:
    def get_fhat(
            fishers: np.ndarray) -> [np.ndarray, np.ndarray]:
        """
        This method computes the normalized Fisher Information Matrix (f_hat) out of a and extracts its trace.
        Args:
            fishers: The FIM to be normalized
        Returns:
             f_hat: The normalized FIM, of size (num_inputs, d, d)
             fisher_trace: The trace of the FIM (before normalizing)
        """

        # compute the trace with all fishers
        fisher_trace = np.trace(np.average(fishers, axis=0))

        # average the fishers over the num_inputs to get the empirical fishers
        fisher_avg = np.average(np.reshape(fishers, (len(fishers), 1, 3, 3)), axis=0)

        # calculate f_hats for all the empirical fishers
        f_hat = 3 * fisher_avg / fisher_trace
        return f_hat, fisher_trace

In [12]:
#https://github.com/ElePT/qiskit-machine-learning/blob/cceaa7e38702e030cf26a1a49f10c12b95b400be/qiskit_machine_learning/algorithms/effective_dimension.py#L174
#goal is to implement this local effective dimension class using the fishers we've already calculated
sample = np.array(iris2[0], requires_grad=True)
mt_fn2 = np.array([np.zeros((3,3))], requires_grad = True)

#calculate fisher for length of inputs
for i in range(len(params)):
    fisher = qml.qinfo.quantum_fisher(circ)(params[i])
    mt_fn2 = np.append(mt_fn2, [fisher],axis = 0)

#delete initialization matrix
mt_fn2 = np.delete(mt_fn2, 0, 0)


In [13]:
led = get_eff_dim(mt_fn2)
print(led)

1.5049248646869626
